<a href="https://colab.research.google.com/github/jpvicencio/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
ave_4br = spark.sql("""
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 4
GROUP BY
    year
ORDER BY
    year;
""")
# Print the results
ave_4br.show()



+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
ave_3br_3ba = spark.sql("""
SELECT
    YEAR(date_built) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND bathrooms = 3
GROUP BY
    year
ORDER BY
    year;
""")

# Print the results
ave_3br_3ba.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2010|    292859.62|
|2011|    291117.47|
|2012|    293683.19|
|2013|    295962.27|
|2014|    290852.27|
|2015|     288770.3|
|2016|    290555.07|
|2017|    292676.79|
+----+-------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
ave_3br_3ba_2flr = spark.sql("""
SELECT
    YEAR(date_built) AS year,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY
    year
ORDER BY
    year
""")
# Print the results
ave_3br_3ba_2flr.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
# Start the timer
start_time = time.time()

# Execute the query
average_price_per_view = spark.sql("""
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view_rating DESC
""")

# Stop the timer
end_time = time.time()

# Print the results
average_price_per_view.show()

# Calculate and print the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime:.2f} seconds")

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
|         80|    991767.38|
+-----------+-------------+
only showing top 20 rows

Runtime: 0.10 seconds
--- 1.0858244895935059 seconds ---


In [9]:

spark.catalog.cacheTable

<bound method Catalog.cacheTable of <pyspark.sql.catalog.Catalog object at 0x79d2a0113dc0>>

In [10]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")


In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
# Start the timer for the cached query
start_time_cached = time.time()

# Execute the query using the cached data
average_price_per_view_cached = spark.sql("""
SELECT
    view AS view_rating,
    ROUND(AVG(price), 2) AS average_price
FROM
    home_sales
GROUP BY
    view
HAVING
    AVG(price) >= 350000
ORDER BY
    view_rating DESC
""")

# Stop the timer for the cached query
end_time_cached = time.time()

# Print the results for the cached query
average_price_per_view_cached.show()

# Calculate and print the runtime
runtime_cached = end_time_cached - start_time_cached
print(f"Cached Query Runtime: {runtime_cached:.2f} seconds")
print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
|         80|    991767.38|
+-----------+-------------+
only showing top 20 rows

Cached Query Runtime: 0.08 seconds
--- 3.870499849319458 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("Resources/home_sales_parquet")


In [19]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.format("parquet").load("Resources/home_sales_parquet")

In [20]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [23]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

# Start the timer for the cached query
parquet_start_time = time.time()

# Execute the query using the cached data
query = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales_parquet
WHERE price >= 350000
GROUP BY view
ORDER BY view
"""

result_df = spark.sql(query)

# Show the results
result_df.show()

# Determine the runtime
parquet_end_time = time.time()
parquet_execution_time = parquet_end_time - parquet_start_time
print(f"Execution time for parquet query: {parquet_execution_time:.2f} seconds")
print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    403848.51|
|   1|    401044.25|
|  10|    401868.43|
| 100|    1026669.5|
|  11|    399548.12|
|  12|    401501.32|
|  13|    398917.98|
|  14|    398570.03|
|  15|     404673.3|
|  16|    399586.53|
|  17|    398474.49|
|  18|    399332.91|
|  19|    398953.17|
|   2|    397389.25|
|  20|    399522.81|
|  21|    399758.88|
|  22|    402022.68|
|  23|    403411.92|
|  24|    400284.92|
|  25|    401298.69|
+----+-------------+
only showing top 20 rows

Execution time for parquet query: 1.11 seconds
--- 372.73833298683167 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached? {is_cached}")

Is home_sales cached? False
